Assignment 7

Name : Summan Bahadur
Roll Number : MSDSF21M509

Using MobileNetV2 , allready trained model for classification of MNIST dataset

In [9]:
import torch
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models.mobilenet import mobilenet_v2
from torch.optim.lr_scheduler import StepLR
from torch.nn import CrossEntropyLoss
import pandas as pd


def train(model, device, train_loader, optimizer, epoch):
    log_interval = 10
    loss_func = CrossEntropyLoss()
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.repeat(1, 3, 1, 1)
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_func(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def tst(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    loss_func = CrossEntropyLoss()
    with torch.no_grad():
        for data, target in test_loader:
            data = data.repeat(1, 3, 1, 1)
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_func(output, target)
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


def main():
    batch_size = 1000
    learning_rate = 1.0
    reduce_lr_gamma = 0.7
    epochs = 4
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print('Device: {} Epochs: {} Batch size: {}'.format(device, epochs, batch_size))

    kwargs = {'batch_size': batch_size}
    if torch.cuda.is_available():
        kwargs.update({'num_workers': 1, 'pin_memory': True})

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    dataset1 = datasets.MNIST('data/', train=True, download=True, transform=transform)
    dataset2 = datasets.MNIST('data/', train=False, transform=transform)
    print('Length train: {} Length test: {}'.format(len(dataset1), len(dataset2)))
    
    train_loader = torch.utils.data.DataLoader(dataset1, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, shuffle=False, **kwargs)
    print('Number of train batches: {} Number of test batches: {}'.format(len(train_loader), len(test_loader)))

    model = mobilenet_v2(pretrained=True)
    model.classifier[1] = torch.nn.Linear(in_features=model.classifier[1].in_features, out_features=10)
    model.to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=learning_rate)

    scheduler = StepLR(optimizer, step_size=1, gamma=reduce_lr_gamma)
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        tst(model, device, test_loader)
        scheduler.step()

    torch.save(model.state_dict(), "mnist_cnn.pt")

    # Final prediction
    ids = list(range(len(dataset2)))
    submission = pd.DataFrame(ids, columns=['id'])
    predictions = []
    real = []
    for data, target in test_loader:
        data = data.repeat(1, 3, 1, 1)
        data = data.to(device)
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        predictions += list(pred.cpu().numpy()[:, 0])
        real += list(target.numpy())
    submission['pred'] = predictions
    submission['real'] = real
    submission.to_csv('submission.csv', index=False)
    print('Submission saved in: {}'.format('submission.csv'))


if __name__ == '__main__':
    main()

Device: cpu Epochs: 4 Batch size: 1000


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST\raw\train-images-idx3-ubyte.gz to data/MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST\raw\train-labels-idx1-ubyte.gz to data/MNIST\raw



...

  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST\raw\t10k-labels-idx1-ubyte.gz to data/MNIST\raw

Length train: 60000 Length test: 10000
Number of train batches: 60 Number of test batches: 10


C:\Users\Summan Bahadur\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\Summan Bahadur\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\Summan Bahadur/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.481981
Train Epoch: 1 [10000/60000 (17%)]	Loss: 1.348076
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.510171
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.103787
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.277915
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.111878

Test set: Average loss: 0.0686, Accuracy: 9801/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.045531
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.037837
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.033032
Train Epoch: 2 [30000/60000 (50%)]	Loss: 0.091579
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.030533
Train Epoch: 2 [50000/60000 (83%)]	Loss: 0.025997

Test set: Average loss: 0.0658, Accuracy: 9817/10000 (98%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.044595
Train Epoch: 3 [10000/60000 (17%)]	Loss: 0.016077
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.026977
Train Epoch: 3 [30000/60000 (50%)]	Loss: 0.023393
Train Epoch: 3 [40000/60000 (67%)]	Loss: 0.011292
Train Epoch: 3 [50000/60000 (83%)]	Loss: 0.